In [1]:
import pandas as pd
import duckdb

176. Вторая по величине зарплата  
Напишите решение для поиска второй по величине уникальной заработной платы в Employeeтаблице. Если второй по величине заработной платы нет, верните значение  null (return None in Pandas).

In [2]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})
employee.head()

,id,salary
0,1,100
1,2,200
2,3,300


In [17]:
data = [[1, 0], [2, 0], [3, 1]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})
employee.head()

,id,salary
0,1,0
1,2,0
2,3,1


In [ ]:
# мое решение
employee = employee.drop_duplicates(subset=['salary']) 
employee['rank'] = employee['salary'].rank(method='dense', ascending=False).astype('int64')
data_1 = [1,2]
ranke_1 = pd.DataFrame(data_1, columns=['ranke_1']).astype({'ranke_1':'int64'})
union_table = ranke_1.merge(employee, left_on='ranke_1', right_on='rank', how='left')
union_table[union_table['ranke_1'] == 2][['salary']].rename(columns={'salary': 'SecondHighestSalary'})

In [ ]:
# решение GPT
salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)
result = (
    salaries.iloc[1:2]
    .to_frame(name='SecondHighestSalary')
)
if result.empty:
    result = pd.DataFrame({'SecondHighestSalary': [None]})

,SecondHighestSalary
0,0


In [19]:
query = """
with t1 as (
    select 
        salary,
        dense_rank() over(order by salary desc) as rank
    from employee
    )
select
    case when count(*) < 1 then null
         else max(salary) end as SecondHighestSalary
from t1
where rank > 1
"""
result = duckdb.query(query).to_df()
result

,SecondHighestSalary
0,0


177. N-я самая высокая зарплата  
Напишите решение для поиска самой высокой уникальной заработной платы в таблице. Если уникальных зарплат  меньше, чем указано , верните значение .nthEmployeennull

In [8]:
N = 2
col_name = f'getNthHighestSalary({N})'
salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)
if N > 0 and len(salaries) >= N:
    result = salaries.iloc[N-1:N].to_frame(name=col_name)
else:
    result = pd.DataFrame({col_name: [None]})
result

,getNthHighestSalary(2)
1,200


In [ ]:
N = 2
query = """
with t1 as (
    select 
        e.salary,
        dense_rank() over(order by e.salary desc) as rank
    from employee e
    )
select
    case when count(*) < 1 or N <= 0 then null
        else max(t1.salary) end as SecondHighestSalary
from t1
where rank > N - 1
"""
result = duckdb.query(query).to_df()
result

178. Ранговые баллы  
Напишите решение для определения ранга оценок. Ранжирование должно рассчитываться в соответствии со следующими правилами:  
- Оценки следует располагать в порядке убывания.  
- Если между двумя результатами есть ничья, оба показателя должны иметь одинаковый рейтинг.  
- В случае ничьей следующий номер в рейтинге должен быть следующим целым числом. Другими словами, между рангами не должно быть разрывов.  
Возвращает таблицу результатов, отсортированную в scoreпорядке убывания.

In [4]:
data = [[1, 3.5], [2, 3.65], [3, 4.0], [4, 3.85], [5, 4.0], [6, 3.65]]
scores = pd.DataFrame(data, columns=['id', 'score']).astype({'id':'Int64', 'score':'Float64'})
scores

,id,score
0,1,3.5
1,2,3.65
2,3,4.0
3,4,3.85
4,5,4.0
5,6,3.65


In [7]:
query = """
select
    score,
    dense_rank() over (order by score desc) as rank
from 
scores
"""
result = duckdb.query(query).to_df()
result

,score,rank
0,4.00,1
1,4.00,1
2,3.85,2
3,3.65,3
4,3.65,3
5,3.50,4


In [11]:
scores['rank'] = scores['score'].rank(method='dense', ascending=False).astype('Int64')
scores.sort_values('rank')[['score', 'rank']]

,score,rank
2,4.0,1
4,4.0,1
3,3.85,2
1,3.65,3
5,3.65,3
0,3.5,4


180. Последовательные номера  
Найдите все числа, которые встречаются как минимум три раза подряд.

In [12]:
data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
logs = pd.DataFrame(data, columns=['id', 'num']).astype({'id':'Int64', 'num':'Int64'})
logs

,id,num
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,2
6,7,2


In [ ]:
numbers = logs['num']

0    1
1    1
2    1
3    2
4    1
5    2
6    2
Name: num, dtype: Int64

In [30]:
result = []
a = numbers[0]
b = 0
for i in range(1, len(numbers)):
    i = numbers[i]
    if i == a:
        b = b + 1
    a = i
    if b >= 3:
        result.append(numbers[i])
result_df = pd.DataFrame(result, columns=['ConsecutiveNums'])
result_df[['ConsecutiveNums']].drop_duplicates()

,ConsecutiveNums
0,1


In [36]:
numbers = logs['num']
if len(numbers) > 0:
    result = []
    a = numbers[0]
    b = 0
    for i in range(1, len(numbers)):
        i = numbers[i]
        if i == a:
            b = b + 1
        a = i
        print(b)
        if b >= 3:
            result.append(numbers[i])
            b = 0
else:
    result = []
result_df = pd.DataFrame(result, columns=['ConsecutiveNums'])
result_df[['ConsecutiveNums']].drop_duplicates()

1
2
2
2
2
3


,ConsecutiveNums
0,1
